# Essentially, testing out my pipeline

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os, pickle
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread

from scripts.utils import extract_frames_info
from scripts.display_utils import show_image_with_bbs
from scripts.hog.classifier import HOGClassifier


In [ ]:
frames_info = extract_frames_info("data/train.csv")

In [ ]:
classifier = HOGClassifier()

features, labels = classifier.features_labels(frames_info[:10], 1)

In [ ]:
features[1].shape

In [ ]:
with open("models/hog/non_linear_stricter_64_8.pkl", "rb") as pickle_file:
    classifier = pickle.load(pickle_file)

In [ ]:
image_index = 100
image = imread(frames_info[image_index][0])

car_index = 5

car_bb = frames_info[image_index][1][car_index]
car_image = image[car_bb[1]: car_bb[1]+car_bb[3], car_bb[0]: car_bb[0]+car_bb[2]]
plt.imshow(car_image)
print(classifier.predict(car_image))

In [ ]:
image_index = 100
image = imread(frames_info[image_index][0])

car_bb = [500, 250, 150, 100]
car_image = image[car_bb[1]: car_bb[1]+car_bb[3], car_bb[0]: car_bb[0]+car_bb[2]]
print(classifier.predict(car_image))
show_image_with_bbs(frames_info[image_index][0], [car_bb])

## Testing detection

In [ ]:
from scripts.hog.detection import detect, nms
import pickle
from skimage.io import imread
from scripts.display_utils import show_image_with_bbs

In [ ]:
test_dir = "data/test"
test_images = [os.path.join(test_dir, filename) for filename in os.listdir(test_dir)]

In [ ]:
image_index = 40
image = imread(test_images[image_index])

In [ ]:
detections, decisions = detect(
    classifier,
    image, 
    10, 
    [
        64,
        100, 
        200, 
        300,
        # 500
    ], 
    [
        # 1.0, 
        .8, 
        .5
    ])
print(len(detections))

In [ ]:
best_idx = np.array(decisions) > .2
best_detections = np.array(detections)[best_idx]
best_decisions = np.array(decisions)[best_idx]

In [ ]:
show_image_with_bbs(test_images[image_index], best_detections, confidences=[f"{score:.3f}" for score in best_decisions])

In [ ]:
print(decisions)

## Hard example mining

In [ ]:
from scripts.hog.bounding_box_utils import find_free_window
from random import choice
from tqdm.auto import tqdm

In [ ]:
WINDOW_WIDTHS, WINDOW_RATIOS = [64, 128, 256, 400], [.8, 0.5, 1.0]

WINDOW_SHAPES = [(width, int(width*ratio)) for width in WINDOW_WIDTHS for ratio in WINDOW_RATIOS]
def mine_hard_example(image_path, bboxes, n_examples, classifier):
    pbar = tqdm()
    examples = []
    image = imread(image_path)
    while len(examples) < n_examples:
        window = find_free_window((image.shape[1], image.shape[0]), bboxes, choice(WINDOW_SHAPES))
        if window is None:
            continue
        sub_image = image[window[1]: window[1]+window[3], window[0]: window[0]+window[2]]
        label, feature_vect = classifier.predict(sub_image, True)
        if label:
            examples.append(feature_vect)
            plt.imshow(sub_image)
            plt.show()
            pbar.update()
    pbar.close()
    return examples

In [ ]:
car_index = 0
image_path, bboxes = frames_info[car_index]

hard_examples = mine_hard_example(image_path, bboxes, 3, classifier)
print(len(hard_examples))

## Visualizing final results

In [ ]:
from ipywidgets import interact, widgets
from scripts.display_utils import show_image_with_bbs
import pickle
import numpy as np
from scripts.hog.detection import detect, nms

In [ ]:
with open("results/hog/linear_results.pkl", "rb") as pickle_file:
    results = pickle.load(pickle_file)


all_filenames, all_detections, all_confidences = [None]*len(results), [None]*len(results), [None]*len(results)
for fname, _, detections, confidences in results:
    idx = int(fname.split(".")[0]) - 1
    all_filenames[idx] = fname
    all_detections[idx] = detections
    all_confidences[idx] = confidences

In [ ]:
def f_display(index):
    detections = all_detections[index]
    confidences = all_confidences[index]
    best_idx = np.array(confidences) > .2
    best_detections = np.array(detections)[best_idx]
    # best_confidences = [f"{score:.3f}" for score in np.array(confidences)[best_idx]]
    best_detections = nms(best_detections, "confidence", .5, "min", np.array(confidences)[best_idx])
    show_image_with_bbs(os.path.join("data/test", all_filenames[index]), best_detections)

interact(f_display, index=widgets.IntSlider(min=0, max=len(all_filenames)-1, step=1, value=83))

In [ ]:
from scripts.utils import run_length_encoding, bounding_boxes_to_mask
import csv

rows = []
for file_name, _, detections, confidences in results:
    correct = [i for i in range(len(detections)) if detections[i][1] > 100 and detections[i][1]+detections[i][3] < 500]
    detections = np.array(detections)[correct]
    confidences = np.array(confidences)[correct]
    best_idx = np.array(confidences) > .2
    best_detections = np.array(detections)[best_idx]
    best_detections = nms(best_detections, "confidence", .5, "min", np.array(confidences)[best_idx])
    if len(best_detections) == 0:
        rle = ""
    else:
        rle = run_length_encoding(bounding_boxes_to_mask(best_detections, 720, 1280))
    rows.append(['test/' + file_name, rle])

with open("results/hog/not_the_worst.csv", "w") as dest_file:
    writer = csv.writer(dest_file)
    writer.writerows(rows)